FACTS: <br>
Number of images in training set: N = 416 <br>
Number of Pixels per image: D = 2576 <br>
Number of classes: c = 52 <br>
-> <br>
Sw: rank(Sw)= N-c = 364 -> Mpca <= 364 but afte <br>
Sb: rank(Sb)= c-1 = 51 -> Mlda <= 51 <br>

In [1]:
# Imports
import scipy.io as sio
import numpy as np
import matplotlib.pyplot as plt

# Load the source file with the face data
mat_content = sio.loadmat('face.mat')

In [2]:
# Store face data and classification
face_data = mat_content['X']
face_label = mat_content['l'] # Not actually used for PCA training, needed for testing

# Append the matching data and labels into array 'faces': unnecessary for PCA training as the labels are not used (unsupervised learning)
# But we need it later for assessing our algorithms during testing
faces = np.append(face_data, face_label, axis=0)

#Transpose in preparation for division into training and testing material
faces_transposed = faces.T

In [3]:
#Divide the data set into training and testing sets: 80% is training, 20% is testing
np.random.shuffle(faces_transposed) # Shuffle the rows (This is why we transposed)
training, test = faces_transposed[:416,:], faces_transposed[416:,:]


training, testing: axis 0: pictures, axis 1: pixels + label

In [26]:
#order training data depending on their class
training = training[np.argsort(training[:,-1]),:]

# Create a list of arrays depending on the classes
class_sets = []
for i in range(52):
    class_sets.append(training[i==(training[:,-1]-1),:2576])
    
print(class_sets[0],class_sets[0].shape)

# Calculate the class means
#class_means = []
#for c in class_sets:
#    class_means.append(np.mean(c, axis=0))
#print(class_means[0],class_means[0].shape)

class_means = np.zeros((52,2576))
for i in range(52):
    class_means[i,:] = np.mean(class_sets[i], axis=0)

# Calculate the global mean
global_mean = np.mean(training[:,:2576], axis=0)
print(global_mean,global_mean.shape)

#Calculate Sb
class_means_norm = class_means - global_mean #make us of broadcasting
Sb = np.dot(class_means_norm.T, class_means_norm)
#print(np.linalg.matrix_rank(Sb)) # =c-1


[[ 73  83 124 ...  33  30  26]
 [191 185 174 ...  22  19  17]
 [106 103 103 ...  37  36  41]
 ...
 [ 93  95  81 ...  88 100  91]
 [ 66  72  67 ... 167 166 163]
 [130 129 130 ...  40  36  35]] (8, 2576)
[121.74278846 121.75240385 119.42788462 ...  87.45192308  85.14423077
  82.85576923] (2576,)
(2576, 2576)
51


In [27]:
#Calculate x-mi
class_sets_norm = []
for i in range(52):
    class_sets_norm.append(class_sets[i]-class_means[i,:])
print(class_sets_norm[0].shape)

(8, 2576)


In [30]:
#Calculate Sw
Sw = np.zeros((2576,2576))
for c in class_sets_norm:
    Sw += np.dot(c.T,c)
    
#print(np.linalg.matrix_rank(Sw)) # = N-c

364
